# 内置图分析算法

图分析在真实世界中有广泛的用处。许多算法比如社区发现，路径连通性，中心性算法都被证明在许多商业中非常有用。

GraphScope 内置了一系列算法，使得用户可以方便的在图数据上做分析。

这个教程展示了如何使用内置算法来完成图分析任务。

## 准备工作

我们首先创建一个会话并且载入一张图。

这里我们使用一个人与人之间的数据集，从 [Gnutella peer-to-peer network, August 31 2002](http://snap.stanford.edu/data/p2p-Gnutella31.html) 中修改而来, 为点和边加入了属性。
这些数据位于 `/testingdata/property`.

In [1]:
import graphscope
from graphscope.framework.loader import Loader

k8s_volumes = {
    "data": {
        "type": "hostPath",
        "field": {
          "path": "/testingdata",  # Path in host
          "type": "Directory"
        },
        "mounts": {
          "mountPath": "/home/jovyan/datasets",
            "readOnly": True
        }
    }
}

graphscope.set_option(show_log=True)  # enable logging
sess = graphscope.session(k8s_volumes=k8s_volumes)

graph = sess.load_from(
    edges={
      "knows": Loader("/home/jovyan/datasets/property/p2p-31_property_e_0", header_row=True)
    },
    vertices={
      "person": Loader("/home/jovyan/datasets/property/p2p-31_property_v_0", header_row=True)
    },
    directed=False,
    generate_eid=False,
  )

来看一下图的数据模型。

In [9]:
print(graph.schema)

oid_type: int64_t
vid_type: uint64_t
label: person
type: VERTEX
properties: [('weight', 'LONG'), ('id', 'LONG')]

label: knows
type: EDGE
properties: [('src_label_id', 'LONG'), ('dst_label_id', 'LONG'), ('dist', 'LONG')]
relations: [('person', 'person')]




如上所示，我们载入了一张属性图，点标签为 `person`，有两个属性 `weight` 和 `id`，边标签为 `knows`， 有三个属性，`src_label_id`, `dst_label_id`, `dist`。

## 投影为简单图

许多图分析算法都只能查询 **简单图**， 在这里我们定义 **简单图** 为只包含一种点和一种边，且点和边最多只有一个属性。

`GraphScope` 提供了一个函数 `project_to_simple` 来将属性图投影为简单图，我们可以选择某一种点和边，以及其属性，来获得属性图的一个 **投影**。

In [10]:
simple_graph = graph.project_to_simple(v_label="person", e_label="knows", v_prop=None, e_prop="dist")

I0128 13:36:46.874050    44 grape_instance.cc:159] Projecting graph, dst graph name: graph_PIhMFSv8, type sig: d41cfdc240dc1887bac2af3abe6ff92626a1ec23fd674eb1cca8da55db3d4b63
I0128 13:36:46.885710    48 grape_instance.cc:159] Projecting graph, dst graph name: graph_PIhMFSv8, type sig: d41cfdc240dc1887bac2af3abe6ff92626a1ec23fd674eb1cca8da55db3d4b63


## 运行算法

在接下来的部分，我们将运行几个算法，并查看结果。

### 单源最短路径

单源最短路径算法（Single Source Shortest Path，接下来将以 `sssp` 代称），需要两个参数，第一个参数是 `graph`，第二个参数是查询的出发点 `src`。

在这里，我们将查询从 ID 为 6 的点出发到所有点的最短路径长度。

在后端，GraphScope 会生成一个兼容被查询的图的算法，编译为一个动态链接库。额外的 GraphScope 会对类型做一些检查，比如，在这里 `sssp` 算法要求边有一个 `int` 或 `double` 的属性作为距离。

第一次编译动态链接库时需要一些时间，但是对于同一个算法，这一步在同样类型的图上只会进行一次。

In [13]:
from graphscope import sssp
sssp_context = sssp(simple_graph, src=6)

2021-01-28 13:46:57,883 [INFO][utils:131]: Codegened application type: cpp_pie, app header: sssp/sssp.h, app_class: grape::SSSP<_GRAPH_TYPE>, vd_type: None, md_type: None, pregel_combine: None
2021-01-28 13:46:57,883 [INFO][utils:135]: Codegened graph type: gs::ArrowProjectedFragment<int64_t,uint64_t,grape::EmptyType,int64_t>, Graph header: core/fragment/arrow_projected_fragment.h
2021-01-28 13:46:57,884 [INFO][utils:187]: Building app ...
I0128 13:47:22.588608    44 parallel_worker.h:87] [Coordinator]: Finished Init
I0128 13:47:22.656388    44 parallel_worker.h:101] [Coordinator]: Finished PEval
I0128 13:47:22.666712    44 parallel_worker.h:115] [Coordinator]: Finished IncEval - 1
I0128 13:47:22.752676    44 parallel_worker.h:115] [Coordinator]: Finished IncEval - 2
I0128 13:47:22.758292    44 parallel_worker.h:115] [Coordinator]: Finished IncEval - 3
I0128 13:47:22.853010    44 parallel_worker.h:115] [Coordinator]: Finished IncEval - 4
I0128 13:47:22.859604    44 parallel_worker.h:11

完成计算后，计算结果分布式的存储到了集群上 `vineyard` 的实例上。

算法会返回一个 `Context` 对象，包含若干种取回结果的方法。

关于 `Context` 的更多信息，请参照 [Context](https://graphscope.io/docs/reference/context.html)

在这里，结果表示从起始点出发到所有点的最短路径，我们使用返回的对象来取得一部分结果，以及取回点ID一并展示。

In [15]:
sssp_context.to_dataframe(selector={'id': 'v.id', 'dist': 'r'}, vertex_range={'begin': 1, 'end': 10}).sort_values(by='id')

,id,dist
4,1,0.0
0,2,8.0
5,3,50.0
1,4,74.0
6,5,59.0
2,6,31.0
7,7,45.0
3,8,45.0
8,9,79.0


另外， 我们还可以将结果输出到客户端的机器的文件系统上。

In [ ]:
sssp_context.output_to_client('./sssp_result.csv', selector={'id': 'v.id', 'dist': 'r'})

查看一下输出的文件。

In [ ]:
!head ./sssp_result.csv

### PageRank

PageRank 是非常著名的一个图分析算法，让我们来看一下如何仅用两行计算 PageRank。

In [16]:
from graphscope import pagerank
pr_context = pagerank(simple_graph, delta=0.85, max_round=10)

2021-01-28 13:57:36,659 [INFO][utils:131]: Codegened application type: cpp_pie, app header: pagerank/pagerank_auto.h, app_class: grape::PageRankAuto<_GRAPH_TYPE>, vd_type: None, md_type: None, pregel_combine: None
2021-01-28 13:57:36,660 [INFO][utils:135]: Codegened graph type: gs::ArrowProjectedFragment<int64_t,uint64_t,grape::EmptyType,int64_t>, Graph header: core/fragment/arrow_projected_fragment.h
2021-01-28 13:57:36,661 [INFO][utils:187]: Building app ...
I0128 13:58:00.935423    44 auto_worker.h:101] [Coordinator]: Finished PEval
I0128 13:58:00.996706    44 auto_worker.h:115] [Coordinator]: Finished IncEval - 1
I0128 13:58:01.087052    44 auto_worker.h:115] [Coordinator]: Finished IncEval - 2
I0128 13:58:01.096709    44 auto_worker.h:115] [Coordinator]: Finished IncEval - 3
I0128 13:58:01.191363    44 auto_worker.h:115] [Coordinator]: Finished IncEval - 4
I0128 13:58:01.287045    44 auto_worker.h:115] [Coordinator]: Finished IncEval - 5
I0128 13:58:01.387101    44 auto_worker.h:1

In [17]:
pr_context.to_dataframe(selector={'id': 'v.id', 'rank': 'r'}, vertex_range={'begin': 1, 'end': 10}).sort_values(by='id')

,id,dist
0,2,0.000033
1,4,0.000043
2,6,0.000017
3,8,0.000017
4,1,0.000024
5,3,0.000007
6,5,0.000011
7,7,0.000029
8,9,0.000018


输出结果到本地。

In [ ]:
pr_context.output_to_client('./pagerank_result.csv', selector={'id': 'v.id', 'rank': 'r'})

### 弱联通分量

在图理论中，无向图中的一个分量，有时被称为一个联通分量，代表一个任意两个节点之间都有边的子图，且这个子图没有指向子图外的点的边。

弱联通分量算法（Weakly Connected Components， WCC）计算每个点所属的联通分量。

In [ ]:
from graphscope import wcc
wcc_context = wcc(simple_graph)
wcc_context.to_dataframe(selector={'id': 'v.id', 'cc': 'r'}, vertex_range={'begin': 1, 'end': 10}).sort_values(by='id')

请查阅 [Builtin algorithms](https://graphscope.io/docs/analytics_engine.html#built-in-algorithms) 来获得更多的内置算法的信息，并欢迎试用更多的算法。

最后，关闭会话以释放资源。

In [ ]:
sess.close()